<a href="https://colab.research.google.com/github/kevinrchilders/computational-number-theory/blob/master/cryptography_chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Picturing the Discrete Logarithm Problem

In [ ]:
# Fast powering algorithm and gcd algorithm

def binary(n):
  binary_repn = []
  if n > 1:
    binary_repn = binary(n // 2)
  binary_repn.append(n % 2)
  return binary_repn

def power(g, A, N):
  A = binary(A)
  total=1
  for i in range(len(A)):
    if A[len(A)-i-1]:
      total = total*g % N
    g = (g*g) % N
  return total

def gcd(a, b):
  return a if b==0 else gcd(b, a%b)

In [ ]:
# Plot of an exponential function in R

x = np.arange(0, 3, 0.1)
plt.figure(figsize=(12,8))
plt.plot(x, np.exp(x))

In [ ]:
# Plot of an exponential function in a finite field with 941 elements.

x = np.arange(941)
y = [power(627, i, 941) for i in x]
plt.figure(figsize=(12,8))
plt.scatter(x, y)

# Diffie-Hellman key exchange

In [ ]:
# Example of Diffie-Hellman key exchange

p=941 # Publicly agreed upon prime
g=627 # Publicly agreed upon element of F_p
print('p =', p)
print('g =', g)

a = 347 # Chosen secretly by Alice
b = 781 # Chosen secretly by Bob

# Exchanged publicly
A = power(g, a, p)
B = power(g, b, p)
print('A =', A)
print('B =', B)

alice_key = power(B, a, p)  # Key as computed by Alice in secret
bob_key = power(A, b, p)    # Key as computed by Bob is secret
print('(Key according to Alice =', alice_key, ')')
print('(Key according to Bob =', bob_key, ')')

In [ ]:
# Eve could attempt to solve 390 = 627^a (mod 941) to deduce the key
# These numbers are much to small for security

%%time

i=0
c=1
while c != A:
  i += 1
  c = power(g, i, p)
print('a = ', i)
print('secret key = ', power(B, a, p))

In [ ]:
# This example uses a 50 digit prime
%%time 

p = 55466907138731394249231482240882676455459190111149
g = 27475823498758492384758293474883297347759810489234
print('p =', p)
print('g =', g)

a = 123457489829374957843291023847058708912734897
b = 47819237874848923

# Exchanged publicly
A = power(g, a, p)
B = power(g, b, p)
print('A =', A)
print('B =', B)

alice_key = power(B, a, p)  # Key as computed by Alice in secret
bob_key = power(A, b, p)    # Key as computed by Bob is secret
print('(Key according to Alice =', alice_key, ')')
print('(Key according to Bob =', bob_key, ')')
print('Keys match:', alice_key==bob_key)

In [ ]:
# Eve's attempted attack

i=0
c=1
while c != A:
  i += 1
  if i%100000 == 0:
    print(100*i/p,'percent done!')
  c = power(g, i, p)
print('a = ', i)
print('secret key = ', power(B, a, p))

# Elgamal PKC

In [ ]:
# A small example of Elgamal PKC

p = 467 # Public prime
g = 2   # Public element of F_p
print('p =', p)
print('g =', g)

a = 153             # Alice's secret key
A = power(g, a, p)  # Published publicly
print('A = ', A)

# Bob wants to send the secret message m = 331
m = 331
print('(Secret message to be encrypted by Bob: ', m, ')')
k = 197 # Chosen randomly in F_p
c1 = power(g, k, p)
c2 = (m * power(A, k, p)) % p
print('Encrypted message: (', c1, ',', c2, ')')

# Alice decrypts by computing c1^(-a)*c2 (mod p)
message = (power(c1, p-1-a, p) * c2) % p
print('(Secret message decoded by Alice: ', message, ')')

# Shank's Babystep-Giantstep algorithm

In [ ]:
# A brute force solution to DLP and Shank's Babystep-giantstep solution to DLP

def brute_dlp(g, h, p):
  x=0
  c=1
  while c != h:
    x += 1
    c = (c*g) % p
  return x


def shank(g, h, p):
  n = int(np.floor(np.sqrt(p-1))+1) # An iteger larger than sqrt(p-1)
  a=1                               # First babystep
  b=h                               # First giantstep
  A = []                            # List of babysteps
  B = []                            # List of giantsteps
  u=power(g, p-1-n, p)              # g^(-n) for giantsteps

  # Fill out lists of steps
  for _ in range(n):
    A.append(a)
    B.append(b)
    a = (a*g) % p
    b = (b*u) % p

  # Find a common value on the two lists
  C = sorted(A + B)
  C1 = C[1:] + [C[0]]
  i=0
  while C[i] != C1[i]:
    i += 1
  common = C[i]

  # Find where this common value occus on each list
  i=0
  while A[i] != common:
    i += 1
  j=0
  while B[j] != common:
    j += 1

  # Find a solutions to DLP
  return i + n*j

In [ ]:
%%time
brute_dlp(47123, 574821237, 3267000013)

In [ ]:
power(47123, 92753857, 3267000013) == 574821237

In [ ]:
%%time
shank(47123, 574821237, 3267000013)